## Programming Applied to Economics

Project for course evaluation

Renan Bassoli Diniz - DOT10 (2024)

### <b>Abstract</b>

The primary objective of this work is to explore the tools and methods presented throughout the course Programming Applied to Economics, taught by Professor Gustavo Ramos Sampaio. In addition, it aimed to address a relevant topic with real potential for academic research. The issue explored here was the evaluation of the impact of mobile internet access on the opening and closure of bank branches at the municipal level. For this, panel data were used to estimate fixed and random effects models. The preliminary results do not allow for major conclusions, as expected, but the signs of the coefficients in the random effects model were in line with expectations. The next steps for this work are to make better use of the sample (by incorporating monthly data, for example), improve the identification and specification of the model, and explore other related topics, such as the demand and supply of municipal credit.

### <b>Introduction</b>

To execute the code, a virtual environment (VENV) named prog101 was created using the command python -m venv prog101 in the Windows 11 terminal. Python version 3.9.13 was used. Due to personal preferences, the Anaconda platform was not utilized.

In [ ]:
pip list # List all installed packages

We can notice that the VENV only contains basic packages. Therefore, let's download the packages we will use throughout the work.

In [ ]:
# Install packages
## MS C++ Build Tools required for Windows Users
!pip install basedosdados matplotlib requests bs4 pypdf numpy==1.26.4 sqlalchemy linearmodels

In [ ]:
pip list # List all installed packages

First, we will use the practice of Web Scraping to obtain the latest Banking Economy Report. This document is prepared and published annually by the Central Bank and covers a wide range of information about the Brazilian banking system, from the performance of credit portfolios to competition analyses in the sector.

In [ ]:
import requests
r = requests.get('https://www.bcb.gov.br/publicacoes/relatorioeconomiabancaria') # Send request to remote address

In [ ]:
# Check if the connection attempt was successful
if r.status_code == 200:
    print('Remote access succesfully completed. You can go on to the next steps.')
elif r.status_code == 400:
    print('Bad request.')
elif r.status_code == 403:
    print('Access forbidden.')
elif r.status_code == 404:
    print('Page not found.')
else:
    print('There\'s something wrong with your request or internet connection.')

Now, let's use BeautifulSoup to search for the download link of the report.

In [ ]:
from bs4 import BeautifulSoup as bs
s = bs(r.text, 'html.parser') # Let's try to extract content from site using BS

In [ ]:
s.find_all('a') # Nothing!

We have a problem! The BCB website is built differently than usual. The layout is dynamic and relies on an internal API for page rendering, which occurs after the Status Code 200, making it impossible to obtain this information directly using Requests. Therefore, we have two alternatives: (i) Use Selenium to control the page as if it were a human user; (ii) Access the API directly, as it occurs "in the background" when analyzing the "Network" tab in the browser's Inspect Mode. In this case, without loss of generality, we will proceed with option (ii).

In [ ]:
r = requests.get('https://www.bcb.gov.br/api/servico/sitebcb/relatorioeconomiabancaria/principal?filtro=') # Access API data
if r.status_code == 200:
    print(r.text)

We can observe that from this new address, we can locate the link to obtain the PDF of the report. Now, let's explore the content, but it's important to note that it is in JSON format.

In [ ]:
import json
j = json.loads(r.content) # Converts response into JSON format

In [ ]:
pdf_link = 'https://www.bcb.gov.br' + j['conteudo'][0]['Url'] # Prepare link address
title = j['conteudo'][0]['Titulo'] # Extract publication title

In [ ]:
print(f'Now we\'re going to get {title} from {pdf_link}')

In [ ]:
# Download, open PDF and search "Sumário Executivo"
rb = requests.get(pdf_link) # Download

In [ ]:
from pypdf import PdfReader, generic

# This PDF file has "Bookmarks" to divide sections
# We're going to take advantage from this in order to find Executive Summary

def getAllDestinations(rq, o, level=0):
    dest = []
    for d in o:
        if isinstance(d,generic.Destination):
            pg_num = rq.get_destination_page_number(d) + 1  # Page count starts from 0
            dest.append([d.title, pg_num, level])
        else:
            # We have a list of destinations, so use recursion
            dest = dest + getAllDestinations(rq, d, level+1)
    return dest

# Open and process it on PyPDF2
with open('reb.pdf', 'rb+') as f:
    f.write(rb.content)
    rdr = PdfReader(f) # Reader
    outlines = rdr.outline # get Bookmarks
    dest = getAllDestinations(rdr, outlines) # get Bookmarks destinations

    for d in dest:
        if 'Sumário Executivo' in d:
            sumario_page = d[1]
            break # Prevent multiple values

    sumario_text = rdr.pages[sumario_page - 1].extract_text()

In [ ]:
print('Now, let\'s read the Executive Summary of the Banking Economy Report from the PDF\'s page ' + str(sumario_page) + '.')

In [ ]:
print(sumario_text)

### <b>Literature review</b>

The Brazilian banking system has undergone significant transformations in recent decades, primarily due to the adoption of new technologies and the development of self-service channels. This literature review examines how these technological innovations, such as Pix, internet banking, and mobile banking, have impacted consumer behavior and the operations of financial institutions in Brazil. The analysis covers studies focusing on different aspects of these transformations, including the reduction of in-person transactions at bank branches and the increasing use of digital channels.

<b>Technological Innovations and Their Impact on Banking Services</b>

Technological innovations have played a fundamental role in modernizing banking services in Brazil. Vilhalba and Bernardo (2023) investigate the impact of Pix, a technology introduced by the Central Bank of Brazil, on the accessibility of financial services, highlighting how this digital tool has facilitated financial inclusion and reduced the need for in-person interactions at bank branches​. Additionally, banks have heavily invested in technology to meet consumer demands for faster and more accessible services, as evidenced by the growth in the use of mobile banking and internet banking.

Lucas de Sousa Lima (2024) complements this discussion by examining the impact of Pix on family farmers participating in the Agroamigo Crescer program. The study reveals that after Pix was implemented, there was a significant reduction in in-person transactions at bank branches, especially among farmers residing in municipalities different from their home branches​
. This suggests that Pix not only facilitated financial transactions but also contributed to financial inclusion in rural areas where access to bank branches can be limited.

<b>Evolution of Self-Service Banking</b>

The development of self-service channels, such as ATMs, internet banking, and mobile banking, has profoundly changed the relationship between customers and banks in Brazil. Viana (2018) explores this evolution, showing that the adoption of these channels has enabled greater customer autonomy, allowing them to perform transactions at any time and from any location​. The research also highlights that the digitization of banking services has resulted in significant cost savings for financial institutions, while also requiring organizational restructuring to adapt to new technologies.

Furthermore, Lima (2024) identifies a similar trend among family farmers in the Agroamigo Crescer program. With the introduction of Pix, these farmers began to increasingly use self-service channels outside of bank branches, such as lottery kiosks and ATMs, demonstrating a clear preference for methods that offer greater convenience and lower costs​.

<b>Transformations in Consumer Banking Behavior</b>

The adoption of new technologies in the banking sector has also brought about significant changes in consumer behavior. As observed by Viana (2018), customers are becoming increasingly familiar with complex operations on digital platforms, suggesting a continued growth in the use of self-service channels​. Additionally, Vilhalba and Bernardo (2023) note that Pix has become an essential tool for low-value transactions, highlighting a shift in consumer habits where convenience and speed are highly valued​.

Lima (2024) reinforces this perspective by analyzing the effects of Pix on the transaction habits of family farmers. The research shows that despite the significant increase in digital transactions, in-person transactions are still common, especially in situations where access to digital technologies is limited or where there is a preference for cash transactions​. However, the general trend points to a continuous migration towards digital methods, driven by convenience and the need to reduce costs associated with travel to bank branches.

### <b>Method and data</b>

#### First-time data download

Since Banco dos Dados has a monthly download limit, we're going to save queries into a sqlite database.

In [ ]:
import basedosdados as bd
import pandas as pd
from sqlalchemy import create_engine, text as sql_text
e = create_engine('sqlite:///prog101_sandbox.db')  # Create a local SQLite Database

In [ ]:
# Anatel dataset
df = bd.read_sql(
    query="""SELECT
                ano,
                dados.sigla_uf AS sigla_uf,
                dados.id_municipio AS id_municipio,
                AVG(densidade) as densidade
            FROM `basedosdados.br_anatel_telefonia_movel.densidade_municipio` AS dados
            GROUP BY ano, sigla_uf, id_municipio""",
    billing_project_id='prog101-434414'
)
df.to_sql('mobile_data', e, if_exists='replace')

In [ ]:
# Caged dataset
df = bd.read_sql(
    query="""SELECT
                ano,
                dados.sigla_uf AS sigla_uf,
                dados.id_municipio AS id_municipio,
                SUM(saldo_movimentacao) as saldo
            FROM `basedosdados.br_me_caged.microdados_movimentacao` AS dados
            GROUP BY ano, sigla_uf, id_municipio""",
    billing_project_id='prog101-434414')
df.to_sql('caged', e, if_exists='replace')

In [ ]:
# Old Caged dataset
df = bd.read_sql(query="""
                SELECT 
                    ano, 
                    sigla_uf, 
                    id_municipio,
                    SUM(CASE WHEN admitidos_desligados = '01' THEN 1 ELSE 0 END) 
                    - SUM(CASE WHEN admitidos_desligados = '02' THEN 1 ELSE 0 END) AS saldo
                FROM 
                    `basedosdados.br_me_caged.microdados_antigos` AS dados
                GROUP BY 
                    ano, 
                    sigla_uf, 
                    id_municipio""", billing_project_id='prog101-434414')
df.to_sql('caged_old', e, if_exists='replace')

In [ ]:
# BCB Estban
df = bd.read_sql(
    query="""SELECT
                ano,
                sigla_uf,
                id_municipio,
                COUNT(cnpj_agencia) as agencias
            FROM `basedosdados.br_bcb_estban.agencia` AS dados
            GROUP BY ano, sigla_uf, id_municipio""",
    billing_project_id='prog101-434414')
df.to_sql('estban', e, if_exists='replace')

In [ ]:
# Google Mobility Report
df = pd.read_csv('changes-visitors-covid.csv')
df['Day'] = pd.to_datetime(df['Day'])
df.info()

In [ ]:
df = df[df['Code'] == "BRA"]
df.to_sql('google', e, if_exists='replace')

#### Data transformation

In [ ]:
import pandas as pd

# Create connection with our sqlite3 database
from sqlalchemy import create_engine, text as sql_text
e = create_engine('sqlite:///prog101_sandbox.db') 

##### Anatel
Yearly change in "Densidade de acessos por 100 domicílios"

In [ ]:
# Anatel
df_anatel = pd.read_sql_query(sql_text("SELECT ano, id_municipio, densidade FROM mobile_data"), e.connect())
df_anatel.head()

In [ ]:
df_anatel.sort_values(by=['id_municipio', 'ano'], inplace=True)
df_anatel['densidade_yoy'] = df_anatel.groupby('id_municipio')['densidade'].pct_change() * 100 # Calculate Year-over-Year change

df_anatel.head(30)

In [ ]:
df_anatel.to_sql('treated_anatel', e, if_exists='replace') # Stash transformed data

##### Caged (payroll change)

Since 2020, we have a new formal jobs dataset for Brazil. So we're going to merge old and new datasets.

In [ ]:
df_caged_new = pd.read_sql_query(sql_text("SELECT ano, id_municipio, saldo FROM caged"), e.connect())
df_caged_new.head(20)

In [ ]:
df_caged_old = pd.read_sql_query(sql_text("SELECT ano, id_municipio, saldo FROM caged_old"), e.connect())
df_caged_old.head(20)

In [ ]:
print(df_caged_new.info())
print(df_caged_old.info())

In [ ]:
df_caged = df_caged_old.append(df_caged_new, ignore_index=True).reset_index()
df_caged.head(10)

In [ ]:
df_caged.to_sql('treated_caged', e, if_exists='replace')

##### Estban (branches opening/closure)

In [ ]:
df_estban = pd.read_sql_query(sql_text("SELECT ano, id_municipio, agencias FROM estban WHERE ano >= 2019"), e.connect())
df_estban.head()

In [ ]:
df_estban.sort_values(by=['id_municipio', 'ano'], inplace=True)
df_estban['agencias_yoy'] = df_estban.groupby('id_municipio')['agencias'].pct_change() * 100

df_estban.head(30)

In [ ]:
df_estban.to_sql('treated_estban', e, if_exists='replace')

##### Put it all together (Panel data sample)

In [ ]:
query = sql_text("""
                    SELECT 
                        a.ano, 
                        a.id_municipio, 
                        a.densidade_yoy, 
                        c.saldo, 
                        e.agencias_yoy
                    FROM 
                        treated_anatel a
                    JOIN 
                        treated_caged c 
                    ON 
                        a.ano = c.ano AND a.id_municipio = c.id_municipio
                    JOIN 
                        treated_estban e 
                    ON 
                        a.ano = e.ano AND a.id_municipio = e.id_municipio
                    WHERE a.ano >= 2020;
                """)
df_work = pd.read_sql_query(query, e.connect())
df_work.head()

In [ ]:
# Drop "municipios" with missing data
nan_counties = df_work[df_work.isna().any(axis=1)]['id_municipio'].unique()
df_work = df_work[~df_work['id_municipio'].isin(nan_counties)]
df_work.head()

In [ ]:
print(f'Sample with {len(df_work["id_municipio"].unique())} counties.')

##### Pandemic dummy creation

In [ ]:
# Why 2020 and 2021 only? We'll discuss it next section
df_work['pandemic'] = df_work['ano'].apply(lambda x: 1 if x == 2020 or x == 2021 else 0)
df_work.head()

In [ ]:
df_work.to_sql('work', e, if_exists='replace')

#### Data visualization

In [ ]:
import matplotlib.pyplot as plt

data = {
    "Id in SQL": ["ano", "id_municipio", "agencias_yoy", "densidade_yoy", "saldo", "pandemic"],
    "Description": ["Year", "County", "Branch count (YoY)", "Mobile usage (YoY)", "Payroll change", "Dummy for pandemic years"],
    "Source": ["Base dos Dados", "Base dos Dados/IBGE", "Base dos Dados/Estban (BCB)", "Base dos Dados/Anatel", "Base dos Dados/Caged", "Our World in Data/Google"],
    "Obs.": [len(df_work["ano"].unique()), len(df_work["id_municipio"].unique()), '-', '-', '-', '-'],
}

# Create figure
dfTmp = pd.DataFrame(data)
fig, ax = plt.subplots(figsize=(10, 3))
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=dfTmp.values, colLabels=dfTmp.columns, cellLoc='center', loc='center')

# Styles adjustment
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.2)

# First-row bold
for (i, j), cell in table.get_celld().items():
    if i == 0:
        cell.set_text_props(weight='bold')
plt.show()


We are going to estimate the effect from change in mobile usage to the change of bank branches by city. Also, we are using Caged dataset as a proxy for the economic activity of the city. Also, we implemented a "pandemic" dummy for 2020 and 2021.

##### Why dummy for pandemic was set only for 2020 and 2020?
Officially, the covid-19 pandemic lasted from 2020 to the middle of 2023. But, the idea here is to consider the most critical periods of the pandemic in terms of mobility and economic activity. Indeed, when we analyze Google's mobility data, we see that from 2022 onwards, there is a relative return to normality in mobility. Although banks were considered "essential services," we believe that the fear of contagion, using mobility as a proxy, impacted the consumption channel of banking services.

In [ ]:
df_mobility = pd.read_sql_query(sql_text("SELECT * FROM google"), e.connect())
df_mobility['Day'] = pd.to_datetime(df_mobility['Day'])
df_mobility.info()

In [ ]:
df_mobility.head()

In [ ]:
plt.figure(figsize=(12, 8))

for column in df_mobility.columns:
    if "Trend" in column:
        plt.plot(df_mobility['Day'], df_mobility[column], label=column)

# Vertical line on Dec 31 2021
plt.axvline(pd.to_datetime('2021-12-31'), color='black', linestyle='--')

# Styles config
plt.xlabel('Date')
plt.ylabel('Trend Value')
plt.title('Evolution of Mobility Trends Over Time')
plt.legend(loc='best')
plt.grid(True)

plt.show()

##### Some examples

Now, we're going to select some cities to check the data.

We have chosen São Paulo, Porto Alegre, Cuiabá, Recife, and Manaus — one capital from each region of the country.

In [ ]:
# Extracted from
# https://www.ibge.gov.br/explica/codigos-dos-municipios.php
cod_ibge_selected = {'3550308': 'São Paulo', '4314902': 'Porto Alegre', '5103403': 'Cuiabá', '2611606': 'Recife', '1302603': 'Manaus'}

In [ ]:
def showChart(id, df, cityname):
    dfW = df.copy()
    dfW = dfW[dfW['id_municipio'] == id]
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Plot config
    ax1.bar(dfW['ano'] - 0.15, dfW['agencias_yoy'], width=0.3, label='Branches # (YoY%)', color='red')
    ax1.bar(dfW['ano'] + 0.15, dfW['densidade_yoy'], width=0.3, label='Mobile density (YoY%)', color='orange')
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Percentage Change')
    ax1.grid(color='gray', linestyle='dashed')

    # Legend
    ax1.legend(loc='upper left')

    # X ticks adjustment
    ax1.set_xticks(dfW['ano'])
    ax1.set_xticklabels(dfW['ano'].astype(int))

    plt.title(f'Yearly Comparison of Mobile Density and Branches ({cityname})')
    plt.show()

In [ ]:
for id, cityname in cod_ibge_selected:
    showChart(id, df_work, cityname)

Analyzing these capital cities, the data does not suggest such a direct inverse relationship between cell phone usage and the closure of bank branches. In fact, the credit market and the Brazilian banking system have other transformation factors that may help explain the decisions to open or close branches.

#### Model estimation

In [ ]:
# Define two indexes to data
# Multi-dimensional data
df_work = df_work.set_index(['id_municipio', 'ano'])

In [ ]:
df_work.head(20)

In [ ]:
from linearmodels.panel import PanelOLS, RandomEffects
import statsmodels.api as sm

X = df_work[['densidade_yoy', 'saldo', 'pandemic']] # Explanatory variables
y = df_work['agencias_yoy'] # Outcome variable

In [ ]:
# Fixed effects model
fixed_effects = PanelOLS(y, X, entity_effects=True)
fixed_effects_result = fixed_effects.fit()

# Random effects model
random_effects = RandomEffects(y, X)
random_effects_result = random_effects.fit()

### <b>Results</b>

In [ ]:
print("\nFixed Effects:\n", fixed_effects_result.summary)

In [ ]:
print("\nRandom Effects:\n", random_effects_result.summary)

In [ ]:
from linearmodels.panel import compare

compare_results = compare([fixed_effects_result, random_effects_result])
print(compare_results)

These results do not allow for major conclusions (nor was it expected), but the random effects model, in particular, showed some signs of the expected coefficients: the increase in cell phone use (densidade_yoy) is negatively correlated with the opening of bank branches. Similarly, the coefficient for the pandemic dummy showed a negative sign. During this period, in addition to the lower customer flow in branches, there was also a cost restructuring, which accelerated the closure of bank branches.

### <b>Conclusion</b>

This work aimed to explore the programming tools applied to economics, presented by Prof. Gustavo, through a case study project on the quantitative determinants of bank branch openings and closures by municipality, which are not currently covered in the literature on the subject.

The literature review indicates that technological innovations have promoted substantial transformation in the Brazilian banking system, both in terms of service offerings and consumer behavior. The digitization of services, exemplified by the growth of Pix and self-service channels, is redefining the relationship between banks and customers, making it more flexible and accessible. However, these advancements also present challenges, such as the need to ensure digital inclusion and organizational adaptation to address an increasingly competitive and technologically advanced environment.

Future research should continue to explore the impacts of these innovations, particularly concerning equitable access to financial services and the social and economic implications of these transformations. Additionally, it is crucial to investigate how financial institutions can continue to evolve to meet the growing expectations of consumers in a digitally oriented market.

The preliminary results do not allow for major conclusions (as expected), but the coefficients of the random effects model were in line with expectations. The next steps in this research are to explore monthly data, improve model identification and specification, and seek a more extensive data window. Moreover, the work does not necessarily need to be limited to explaining branch openings and closures. It can also explore the determinants of municipal credit demand and supply.

### <b>References</b>

* CENTRAL BANK OF BRAZIL. Banking Economy Report: 2023. Brasília: Central Bank of Brazil, 2023. Available at: https://www.bcb.gov.br/publicacoes/relatorioeconomiabancaria. Accessed on: September 4, 2024. <br>
* Lima, L. de S. (2024). Saques e Transações Presenciais em Agências Bancárias por Agricultores Familiares e Efeito PIX: O Caso dos Clientes do Agroamigo Crescer. Dissertação de Mestrado, Universidade Federal do Ceará. <br>
* Viana, T. H. P. (2018). Análise do Autoatendimento Bancário no Brasil. Trabalho de Conclusão de Curso (Graduação em Administração), Universidade Federal de Ouro Preto. <br>
* Vilhalba, C. de A., & Bernardo, L. V. M. (2023). O consumo dos serviços bancários, frente as inovações: uma análise dos dados do Banco Central do Brasil. Universidade Federal da Grande Dourados.